In [1]:
from ssd import build_ssd

## box_utils裡面很多Function，可以看看是怎麼設計的
from layers.box_utils import * 
from torch.autograd import Variable
from layers import functions
from layers import modules
from math import sqrt 
from itertools import product
from torch.autograd import Function

import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import argparse
import pickle
import torch.nn.functional as F


In [2]:
## 詳細模型結構可以參考ssd.py
ssd_net=build_ssd('train', size=300, num_classes=21)

C:\Users\lu\Desktop\2nd-DL-CVMarathon\D29-31\Object Detection 程式導讀\ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)


## 默認Config檔案在data/config.py內

In [3]:
ssd_net.cfg

{'num_classes': 21,
 'lr_steps': (80000, 100000, 120000),
 'max_iter': 120000,
 'feature_maps': [38, 19, 10, 5, 3, 1],
 'min_dim': 300,
 'steps': [8, 16, 32, 64, 100, 300],
 'min_sizes': [30, 60, 111, 162, 213, 264],
 'max_sizes': [60, 111, 162, 213, 264, 315],
 'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
 'variance': [0.1, 0.2],
 'clip': True,
 'name': 'VOC'}

In [4]:
cfg = {
    'num_classes': 21,
    'lr_steps': (80000, 100000, 120000),
    'max_iter': 120000,
    'feature_maps': [38, 19, 10, 5, 3, 1],
    'min_dim': 300,
    'steps': [8, 16, 32, 64, 100, 300],
    'min_sizes': [30, 60, 111, 162, 213, 264],
    'max_sizes': [60, 111, 162, 213, 264, 315],
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
    'variance': [0.1, 0.2],
    'clip': True,
    'name': 'VOC',
}

### 'aspect_ratios' : 使用六張Feature Map，每一張上方有預設的anchor boxes，Boxes aspect ratio可以自己設定
### 'feature_maps' : 使用feature map大小為[38x38, 19x19, 10x10, 5x5, 3x3, 1x1]
### 'min_sizes'、'max_sizes'可藉由下方算式算出，由作者自行設計
### 'steps' : Feature map回放回原本300*300的比例，如38要回放為300大概就是8倍
### 'variance' : Training 的一個trick，加速收斂，詳見：https://github.com/rykov8/ssd_keras/issues/53

---

## 'min_sizes'、'max_sizes' 計算

In [5]:
import math
## source:https://blog.csdn.net/gbyy42299/article/details/81235891
min_dim = 300   ## 维度
# conv4_3 ==> 38 x 38
# fc7 ==> 19 x 19
# conv6_2 ==> 10 x 10
# conv7_2 ==> 5 x 5
# conv8_2 ==> 3 x 3
# conv9_2 ==> 1 x 1
mbox_source_layers = ['conv4_3', 'fc7', 'conv6_2', 'conv7_2', 'conv8_2', 'conv9_2'] ## prior_box來源層，可以更改。很多改進都是基於此處的調整。
# in percent %
min_ratio = 20 ## 這裡即是論文中所說的Smin的= 0.2，Smax的= 0.9的初始值，經過下面的運算即可得到min_sizes，max_sizes。
max_ratio = 90
step = int(math.floor((max_ratio - min_ratio) / (len(mbox_source_layers) - 2)))## 取一個間距步長，即在下面用於循環給比取值時起一個間距作用。可以用一個具體的數值代替，這裡等於17。
min_sizes = []  ## 經過以下運算得到min_sizes和max_sizes。
max_sizes = []
for ratio in range(min_ratio, max_ratio + 1, step):
    ## 從min_ratio至max_ratio + 1每隔步驟= 17取一個值賦值給比。注意範圍函數的作用。
    ## min_sizes.append（）函數即把括號內部每次得到的值依次給了min_sizes。
    min_sizes.append(min_dim * ratio / 100.)
    max_sizes.append(min_dim * (ratio + step) / 100.)
min_sizes = [min_dim * 10 / 100.] + min_sizes
max_sizes = [min_dim * 20 / 100.] + max_sizes

## steps: 這一步要仔細理解，即計算卷積層產生的prior_box距離原圖的步長，先驗框中心點的坐標會乘以step，
## 相當於從特徵映射位置映射回原圖位置，比如conv4_3輸出特徵圖大小為38 *38，而輸入的圖片為300* 300，
## 所以38 *8約等於300，所以映射步長為8.這是針對300* 300的訓練圖片。
steps = [8, 16, 32, 64, 100, 300]  
aspect_ratios = [[2], [2, 3], [2, 3], [2, 3], [2], [2]]
 
print('min_sizes: ',min_sizes)
print('max_sizes: ',max_sizes)


min_sizes:  [30.0, 60.0, 111.0, 162.0, 213.0, 264.0]
max_sizes:  [60.0, 111.0, 162.0, 213.0, 264.0, 315.0]


---

## Default anchor boxes設計原理，看懂收穫很多
##### 可以理解 SSD原文中 8732個anchors是怎麼來的
##### 38×38×4+19×19×6+10×10×6+5×5×6+3×3×4+1×1×4=8732

In [6]:
class PriorBox(object):
    """Compute priorbox coordinates in center-offset form for each source
    feature map.
    """
    def __init__(self, cfg):
        super(PriorBox, self).__init__()
        self.image_size = cfg['min_dim']
        # number of priors for feature map location (either 4 or 6)
        self.num_priors = len(cfg['aspect_ratios'])
        self.variance = cfg['variance'] or [0.1]
        self.feature_maps = cfg['feature_maps']
        self.min_sizes = cfg['min_sizes']
        self.max_sizes = cfg['max_sizes']
        self.steps = cfg['steps']
        self.aspect_ratios = cfg['aspect_ratios']
        self.clip = cfg['clip']
        self.version = cfg['name']
        for v in self.variance:
            if v <= 0:
                raise ValueError('Variances must be greater than 0')

    def forward(self):
        mean = []
        '''依照Feature map大小找出所有的pixel 中心'''
        '''下方這兩個loop會找出W個x軸pixel對上W個y軸pixel，假如現在是在38x38的feature map上，就會有38x38個值'''
        '''ex. [0,1],[0,2]..[0,37] [1,1],[1,2]..[1,37]..........[37,37]'''
        for k, f in enumerate(self.feature_maps):
            for i, j in product(range(f), repeat=2):
                f_k = self.image_size / self.steps[k] ## 如self.steps==8，就是先將原圖size normalize(/300)後再乘上8
                # unit center x,y
                '''中心點'''
                cx = (j + 0.5) / f_k
                cy = (i + 0.5) / f_k

                # aspect_ratio: 1
                # rel size: min_size
                '''/self.image_size 就是在做normalization '''
                s_k = self.min_sizes[k]/self.image_size
                '''小的正方形box'''
                mean += [cx, cy, s_k, s_k]

                # aspect_ratio: 1
                # rel size: sqrt(s_k * s_(k+1))
                '''大的正方形box'''
                s_k_prime = sqrt(s_k * (self.max_sizes[k]/self.image_size))
                mean += [cx, cy, s_k_prime, s_k_prime]

                # rest of aspect ratios
                for ar in self.aspect_ratios[k]:
                    '''aspect ratio 2,3'''
                    mean += [cx, cy, s_k*sqrt(ar), s_k/sqrt(ar)]
                    '''aspect ratio 1/2,1/3'''
                    mean += [cx, cy, s_k/sqrt(ar), s_k*sqrt(ar)]
        # back to torch land
        output = torch.Tensor(mean).view(-1, 4)
        if self.clip:
            output.clamp_(max=1, min=0)
        return output

In [7]:
PriorBox_Demo=PriorBox(cfg)

In [8]:
print(PriorBox_Demo.forward().shape)

torch.Size([8732, 4])


---

## Loss 如何設計

In [9]:
class MultiBoxLoss(nn.Module):

    def __init__(self, num_classes, overlap_thresh, prior_for_matching,
                 bkg_label, neg_mining, neg_pos, neg_overlap, encode_target,
                 use_gpu=True):
        super(MultiBoxLoss, self).__init__()
        self.use_gpu = use_gpu
        '''有幾類'''
        self.num_classes = num_classes
        '''判定為正樣本的threshold，一般設為0.5'''
        self.threshold = overlap_thresh
        '''background自己會有一類，不用Label，假如我們有20類一樣標註0-19，下方會自己空出一類給background'''
        self.background_label = bkg_label
        self.encode_target = encode_target
        self.use_prior_for_matching = prior_for_matching
        '''OHEM，找出分得最不好的樣品，也就是confidence score比較低的正負樣品'''
        self.do_neg_mining = neg_mining
        '''負樣品與正樣品的比例，通常是3:1'''
        self.negpos_ratio = neg_pos
        self.neg_overlap = neg_overlap
        self.variance = cfg['variance']
     

    def forward(self, predictions, targets):

        '''prediction會output三個值'''
        '''loc shape: bounding box 資訊，torch.size(batch_size,num_priors,4)'''
        '''conf shape: 每一個bounding box 的信心程度，torch.size(batch_size,num_priors,num_classes)'''
        '''priors shape: 預設的defaul box， torch.size(num_priors,4)'''
        loc_data, conf_data, priors = predictions
        num = loc_data.size(0)
        priors = priors[:loc_data.size(1), :]
        num_priors = (priors.size(0))
        num_classes = self.num_classes

        # match priors (default boxes) and ground truth boxes
        loc_t = torch.Tensor(num, num_priors, 4)
        conf_t = torch.LongTensor(num, num_priors)
        for idx in range(num):
            truths = targets[idx][:, :-1].data
            labels = targets[idx][:, -1].data
            defaults = priors.data
            '''jaccard 計算每一個BBOX與ground truth的IOU'''
            match(self.threshold, truths, defaults, self.variance, labels,
                  loc_t, conf_t, idx)
        if self.use_gpu:
            loc_t = loc_t.cuda()
            conf_t = conf_t.cuda()
        '''用Variable包裝'''
        loc_t = Variable(loc_t, requires_grad=False)
        conf_t = Variable(conf_t, requires_grad=False)

        pos = conf_t > 0
        num_pos = pos.sum(dim=1, keepdim=True)


        pos_idx = pos.unsqueeze(pos.dim()).expand_as(loc_data)
        loc_p = loc_data[pos_idx].view(-1, 4)
        loc_t = loc_t[pos_idx].view(-1, 4)
        '''smooth_l1_loss 計算bounding box regression'''
        loss_l = F.smooth_l1_loss(loc_p, loc_t, size_average=False)

        # Compute max conf across batch for hard negative mining
        batch_conf = conf_data.view(-1, self.num_classes)
        loss_c = log_sum_exp(batch_conf) - batch_conf.gather(1, conf_t.view(-1, 1))

        # Hard Negative Mining
        loss_c = loss_c.view(num, -1)
        loss_c[pos] = 0
        '''排列confidence 的分數'''
        _, loss_idx = loss_c.sort(1, descending=True)
        _, idx_rank = loss_idx.sort(1)
        num_pos = pos.long().sum(1, keepdim=True)
        '''負樣品取出數量 == negpos_ratio*num_pos'''
        num_neg = torch.clamp(self.negpos_ratio*num_pos, max=pos.size(1)-1)
        neg = idx_rank < num_neg.expand_as(idx_rank)

        # Confidence Loss Including Positive and Negative Examples
        pos_idx = pos.unsqueeze(2).expand_as(conf_data)
        neg_idx = neg.unsqueeze(2).expand_as(conf_data)
        conf_p = conf_data[(pos_idx+neg_idx).gt(0)].view(-1, self.num_classes)
        targets_weighted = conf_t[(pos+neg).gt(0)]
        '''用cross_entropy做分類'''
        loss_c = F.cross_entropy(conf_p, targets_weighted, size_average=False)

        # Sum of losses: L(x,c,l,g) = (Lconf(x, c) + αLloc(x,l,g)) / N
        #double轉成torch.float64
        N = num_pos.data.sum().double()
        loss_l = loss_l.double()
        loss_c = loss_c.double()
        loss_l /= N
        loss_c /= N
        return loss_l, loss_c


## 產生我們Loss function，注意這裡的class要包含背景

In [10]:
Use_cuda=False
criterion = MultiBoxLoss(21, 0.5, True, 0, False, 3, 0.5,False, Use_cuda,)

----

## 基本設定

In [11]:
ssd_net=build_ssd('train', size=300, num_classes=21)
use_pretrained=False
if use_pretrained:
    ssd_net.load_weights('./demo/ssd300_mAP_77.43_v2.pth')
net=ssd_net

In [12]:
'''要不要使用gpu'''
Use_cuda=False

'''tensor type會依照cpu或gpu有所不同'''
if torch.cuda.is_available():
    if Use_cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    else:
        print("WARNING: It looks like you have a CUDA device, but aren't " +
              "using CUDA.\nRun with --cuda for optimal training speed.")
        torch.set_default_tensor_type('torch.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')


'''使用GPU時模型要轉成cuda'''
if Use_cuda:
    net = net.cuda()
    
batch_size_=32
optimizer = optim.Adam(net.parameters(),lr=0.00001/batch_size_)

---

## 訓練

## 這裡我們先示範輸入的 image,Label格式，真正在訓練時，準備成一樣格式即可

In [13]:
'''輸入影像格式，假設batch size 為 4'''
image_in=torch.tensor(torch.rand(4,3,300,300),dtype=torch.float32)
'''Label格式，沒有固定長度，看圖像中有幾個label就有幾個'''
label_0=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 19.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]
label_1=[[ 0.1804,  0.6076,  0.7701,  0.8485, 13.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 11.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 7.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 5.0000],]
label_2=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 14.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]
label_3=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 19.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]

C:\Users\lu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [14]:
epochs=300
iteration=1000

In [15]:
for epoch in range(epochs):
    n=0
    loss_sum=[]
    loc_loss=[]
    conf_loss=[]
    for number__ in range(iteration) :
        '''要用Variable包裝tensor才能送入模型'''
        if Use_cuda:
            image_ = Variable(image_in.cuda())
            y = [Variable(torch.tensor(label_0).cuda(), volatile=True),Variable(torch.tensor(label_1).cuda(), 
                volatile=True),Variable(torch.tensor(label_2).cuda(), volatile=True),Variable(torch.tensor(label_3).cuda(), volatile=True)]      
        else:
            image_ = Variable(image_in)
            y = [Variable(torch.tensor(label_0), volatile=True),Variable(torch.tensor(label_1), 
                volatile=True),Variable(torch.tensor(label_2), volatile=True),Variable(torch.tensor(label_3), volatile=True)]

        '''Forward Pass'''
        out = net(image_)
        '''Regression Loss and Classification Loss'''
        loss_l,loss_c = criterion(out,y )
        loss = loss_l+ loss_c
        '''Backward'''
        loss.backward()

        loc_loss.append(loss_l.data.cpu().numpy())
        conf_loss.append(loss_c.data.cpu().numpy())
        loss_sum.append(loss.data.cpu().numpy())
        '''更新參數'''
        optimizer.step()
        '''清空Gradients'''
        optimizer.zero_grad()
        
        n+=1
        if n%10==0:
            print('BBOX Regression Loss: ', np.mean(loc_loss))
            print('Classification Loss: ', np.mean(conf_loss))
    '''儲存權重'''
    torch.save(ssd_net.state_dict(),'weights/Ｗeights.pth')

C:\Users\lu\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
C:\Users\lu\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  from ipykernel import kernelapp as app
C:\Users\lu\Anaconda3\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


BBOX Regression Loss:  2.519275269684968
Classification Loss:  17.188670857747393
BBOX Regression Loss:  2.519151475694444
Classification Loss:  17.07607427526403
BBOX Regression Loss:  2.5190279360170726
Classification Loss:  16.96338986997251
BBOX Regression Loss:  2.5189044528537328
Classification Loss:  16.85041594328704
BBOX Regression Loss:  2.5187809357819737
Classification Loss:  16.737118349428528
BBOX Regression Loss:  2.5186573876274965
Classification Loss:  16.623640215838396
BBOX Regression Loss:  2.5185336884998137
Classification Loss:  16.509963391823742
BBOX Regression Loss:  2.518409902078134
Classification Loss:  16.39625988713017
BBOX Regression Loss:  2.5182860762984665
Classification Loss:  16.282912379135322
BBOX Regression Loss:  2.5181622540509263
Classification Loss:  16.170343831380208
BBOX Regression Loss:  2.518038418959287
Classification Loss:  16.059547152021516
BBOX Regression Loss:  2.51791456717032
Classification Loss:  15.953777416841483
BBOX Regressio

Classification Loss:  12.92942490754304
BBOX Regression Loss:  2.489496753833912
Classification Loss:  12.150622671621816
BBOX Regression Loss:  2.4892116687915946
Classification Loss:  12.150040577076098
BBOX Regression Loss:  2.4889199904453605
Classification Loss:  12.149452190634642
BBOX Regression Loss:  2.4886212242974173
Classification Loss:  12.148857653582537
BBOX Regression Loss:  2.488314875849971
Classification Loss:  12.148252992983219
BBOX Regression Loss:  2.488001058131089
Classification Loss:  12.147639842092255
BBOX Regression Loss:  2.487680259583488
Classification Loss:  12.14701810967985
BBOX Regression Loss:  2.4873529328240287
Classification Loss:  12.14638803270128
BBOX Regression Loss:  2.487018930568617
Classification Loss:  12.145749052071276
BBOX Regression Loss:  2.4866772828278716
Classification Loss:  12.145100583676937
BBOX Regression Loss:  2.486327110316216
Classification Loss:  12.14444272847288
BBOX Regression Loss:  2.4859675442730937
Classification

BBOX Regression Loss:  2.2745036779333043
Classification Loss:  11.776685595477069
BBOX Regression Loss:  1.8921816931830513
Classification Loss:  10.690495469835069
BBOX Regression Loss:  1.8899871896814417
Classification Loss:  10.676453371401186
BBOX Regression Loss:  1.8876483022430794
Classification Loss:  10.662536997853975
BBOX Regression Loss:  1.8852098924142342
Classification Loss:  10.648615886546947
BBOX Regression Loss:  1.8827121310763888
Classification Loss:  10.634609962745948
BBOX Regression Loss:  1.880182374553916
Classification Loss:  10.6204828898112
BBOX Regression Loss:  1.8776343451605904
Classification Loss:  10.60620541850095
BBOX Regression Loss:  1.8750728289286296
Classification Loss:  10.59174673292372
BBOX Regression Loss:  1.87250161033599
Classification Loss:  10.577084727346161
BBOX Regression Loss:  1.8699149237738717
Classification Loss:  10.56221260353371
BBOX Regression Loss:  1.8673184841168848
Classification Loss:  10.547114516749524
BBOX Regress

BBOX Regression Loss:  1.3259846005616367
Classification Loss:  7.256063849837692
BBOX Regression Loss:  0.3919795848705151
Classification Loss:  3.9088891488534436
BBOX Regression Loss:  0.38550074895222985
Classification Loss:  3.892164654201932
BBOX Regression Loss:  0.37917450563407246
Classification Loss:  3.875572986367308
BBOX Regression Loss:  0.37293998488673463
Classification Loss:  3.8592064045093677
BBOX Regression Loss:  0.36680019802517366
Classification Loss:  3.8430156283908423
BBOX Regression Loss:  0.3608599957124687
Classification Loss:  3.8268625094566815
BBOX Regression Loss:  0.35513183250629077
Classification Loss:  3.810716792515346
BBOX Regression Loss:  0.3496384960633737
Classification Loss:  3.7945313029819063
BBOX Regression Loss:  0.3443243065979255
Classification Loss:  3.778340224277826
BBOX Regression Loss:  0.33910227846216273
Classification Loss:  3.7622481762921365
BBOX Regression Loss:  0.33397058782352745
Classification Loss:  3.746257332438973
BBO

...

Classification Loss:  0.8160824711816906
BBOX Regression Loss:  0.002661030478557648
Classification Loss:  0.815153396561829
BBOX Regression Loss:  0.0026360675583149444
Classification Loss:  0.8142424219920312
BBOX Regression Loss:  0.0026116597860889406
Classification Loss:  0.8133480195099728
BBOX Regression Loss:  0.0025877108078524585
Classification Loss:  0.8124734340791709
BBOX Regression Loss:  0.0025643461317679675
Classification Loss:  0.8116025955365741
BBOX Regression Loss:  0.0025413073501549662
Classification Loss:  0.8107517238193088
BBOX Regression Loss:  0.0002537896373757609
Classification Loss:  0.7273709897641782
BBOX Regression Loss:  0.00025410128205462744
Classification Loss:  0.7271441141764321
BBOX Regression Loss:  0.00025375372856671424
Classification Loss:  0.7269353018866644
BBOX Regression Loss:  0.0002567687490092659
Classification Loss:  0.7268383238050673
BBOX Regression Loss:  0.0002656366238025603
Classification Loss:  0.7266786321004233
BBOX Regressi

BBOX Regression Loss:  0.00027095715145576986
Classification Loss:  0.7229333975986794
BBOX Regression Loss:  0.00027141759030275625
Classification Loss:  0.7229074286629634
BBOX Regression Loss:  0.0002716412122437551
Classification Loss:  0.7228816604924775
BBOX Regression Loss:  0.00027184432971211756
Classification Loss:  0.722856079629846
BBOX Regression Loss:  0.00027238164100408773
Classification Loss:  0.7228307332046855
BBOX Regression Loss:  0.00027281108972454304
Classification Loss:  0.7228056132840396
BBOX Regression Loss:  0.000273417791961665
Classification Loss:  0.722780684160675
BBOX Regression Loss:  0.00027358095336958395
Classification Loss:  0.722755946274157
BBOX Regression Loss:  0.0002737937238367517
Classification Loss:  0.7227313751389662
BBOX Regression Loss:  0.00027458130185600046
Classification Loss:  0.7227069905916128
BBOX Regression Loss:  0.00027559673862397293
Classification Loss:  0.7226827661419378
BBOX Regression Loss:  0.0002760739207474722
Class

Classification Loss:  0.7189979531944781
BBOX Regression Loss:  0.0003855349569100184
Classification Loss:  0.7189822373566804
BBOX Regression Loss:  0.00038616237114944484
Classification Loss:  0.7189665118281878
BBOX Regression Loss:  0.00038685580871158584
Classification Loss:  0.7189507223222915
BBOX Regression Loss:  0.0003873930332913405
Classification Loss:  0.7189349512690533
BBOX Regression Loss:  0.00038782331460666676
Classification Loss:  0.7189191385551736
BBOX Regression Loss:  0.0003880443544054572
Classification Loss:  0.718903290029868
BBOX Regression Loss:  0.0003896924640998674
Classification Loss:  0.7188874209368671
BBOX Regression Loss:  0.0003925257514044794
Classification Loss:  0.7188716038388355
BBOX Regression Loss:  0.00039361250240373726
Classification Loss:  0.7188557287729114
BBOX Regression Loss:  0.00039403637224301685
Classification Loss:  0.7188397130658549
BBOX Regression Loss:  0.00039437026566318677
Classification Loss:  0.7188237090294901
BBOX Reg

Classification Loss:  0.7152576176642467
BBOX Regression Loss:  0.0006226158852829126
Classification Loss:  0.715227504073875
BBOX Regression Loss:  0.000623875791595007
Classification Loss:  0.7151972188772979
BBOX Regression Loss:  0.0006248415775672044
Classification Loss:  0.7151666903746526
BBOX Regression Loss:  0.0006275109157164756
Classification Loss:  0.7151360734906839
BBOX Regression Loss:  0.0006333461115681157
Classification Loss:  0.7151054031664855
BBOX Regression Loss:  0.0006357157653414463
Classification Loss:  0.7150742291142702
BBOX Regression Loss:  0.0006369290038667344
Classification Loss:  0.7150427175451207
BBOX Regression Loss:  0.0006379103245940397
Classification Loss:  0.7150109283683835
BBOX Regression Loss:  0.0006388676751653987
Classification Loss:  0.7149788330747199
BBOX Regression Loss:  0.000640035528957731
Classification Loss:  0.7149464689119898
BBOX Regression Loss:  0.0006472829078594273
Classification Loss:  0.71491420371206
BBOX Regression Lo

KeyboardInterrupt: 

## 想要Train VOC,COCO可以參考原Github:https://github.com/amdegroot/ssd.pytorch